# How to call functions with chat models

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`functions` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

If the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `function_call` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `function_call` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_call"` in the response, as well as a `function_call` object that has the name of the function and the generated function arguments.

### Overview

This notebook contains the following 2 sections:

- **How to generate function arguments:** Specify a set of functions and use the API to generate function arguments.
- **How to call functions with model generated arguments:** Close the loop by actually executing functions with model generated arguments.

## How to generate function arguments

In [1]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4"

### Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [3]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


## How to call functions with model generated arguments

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/).

*Note:* SQL generation can be high-risk in a production environment since models are not perfectly reliable at generating correct SQL.

### Specifying a function to execute SQL queries

First let's define some helpful utility functions to extract data from a SQLite database.

In [4]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")

print("Opened database successfully")

Opened database successfully


In [5]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

def execute_query(conn, q):
    return conn.execute(q).fetchall()


In [3]:
print ( execute_query(conn,"select count(*) from artists"))

[(275,)]


In [ ]:
q="""
select g.Name, q.c 
from
  ( select GenreId, count(*) as c
  from "tracks" group by GenreId)q 
inner join genres g
where g.GenreId = q.GenreId 
order by c desc 
"""
print ( execute_query(conn,q))

In [ ]:
### gets the least popular track
q = """
select g.Name, q.c from
 (select GenreId, count(*) as c
  from "tracks" group by GenreId)q
inner join genres g 
where g.GenreId = q.GenreId order by q.c asc limit 1
"""


print ( execute_query(conn, q))

In [ ]:
### gets most prolific artists
q = """
select a.name, q.c 
from artists a inner join
  (select ArtistId, count(*) c
  from albums
  group by ArtistId
  order by c desc)q 
  where a.ArtistId=q.ArtistId;
"""
print ( execute_query(conn, q))

In [23]:
q="""
 select distinct(Name) from genres
"""
print ( execute_query(conn, q))

[('Rock',), ('Jazz',), ('Metal',), ('Alternative & Punk',), ('Rock And Roll',), ('Blues',), ('Latin',), ('Reggae',), ('Pop',), ('Soundtrack',), ('Bossa Nova',), ('Easy Listening',), ('Heavy Metal',), ('R&B/Soul',), ('Electronica/Dance',), ('World',), ('Hip Hop/Rap',), ('Science Fiction',), ('TV Shows',), ('Sci Fi & Fantasy',), ('Drama',), ('Comedy',), ('Alternative',), ('Classical',), ('Opera',)]


In [24]:
q="""
select innerQ.CustomerId, innerQ.cName, count(distinct(innerQ.gName)) genreCount
from 
 (select c.FirstName || ' ' || c.LastName as cName, c.CustomerId, genres.Name as gName
 from invoice_items items
 inner join invoices invs 
 on  invs.InvoiceId=items.InvoiceId
 inner join customers
 c on c.CustomerId=invs.CustomerId
 inner join tracks
 on items.TrackId = tracks.TrackId
 inner join genres on genres.GenreId = tracks.GenreId)innerQ
 group by innerQ.CustomerId
 order by genreCount desc
 limit 5

"""
print ( execute_query(conn, q))

[(57, 'Luis Rojas', 12), (45, 'Ladislav Kovács', 11), (3, 'François Tremblay', 10), (14, 'Mark Philips', 10), (17, 'Jack Smith', 10)]


In [11]:
### which customers have spent the most money
q = """
select i.CustomerId, c.FirstName || ' ' ||  c.LastName as customerName,  round(sum(spendPerInvoice.totalSpent),2) as spendPerCustomer 
from invoices i
inner join
(select InvoiceId, round(sum(UnitPrice * Quantity),2) as totalSpent 
  from invoice_items
  group by InvoiceId 
  order by totalSpent desc) spendPerInvoice
on i.InvoiceId = spendPerInvoice.InvoiceId
inner join customers c
on c.CustomerId=i.CustomerId
group by i.CustomerId
order by spendPerCustomer desc
limit 5

"""
print ( execute_query(conn, q))

[(6, 'Helena Holý', 49.62), (26, 'Richard Cunningham', 47.62), (57, 'Luis Rojas', 46.62), (46, "Hugh O'Reilly", 45.62), (45, 'Ladislav Kovács', 45.62)]


Now can use these utility functions to extract a representation of the database schema.

In [7]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)
print (database_schema_string)

Table: albums
Columns: AlbumId, Title, ArtistId
Table: sqlite_sequence
Columns: name, seq
Table: artists
Columns: ArtistId, Name
Table: customers
Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
Table: employees
Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
Table: genres
Columns: GenreId, Name
Table: invoices
Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
Table: invoice_items
Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
Table: media_types
Columns: MediaTypeId, Name
Table: playlists
Columns: PlaylistId, Name
Table: playlist_track
Columns: PlaylistId, TrackId
Table: tracks
Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice
Table: sqlite_stat1
Columns: tbl, idx, stat


As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [8]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

### Executing SQL queries

Now let's implement the function that will actually excute queries against the database.

In [9]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message["function_call"]["name"] == "ask_database":
        query = json.loads(message["function_call"]["arguments"])["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results

In [10]:
## this method performs the full conversational flow, generating the function, executing it and
## describing the results
def do_conversation(conn, input):
    
    messages = []
    messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})
    messages.append({"role": "user", "content": input})
    chat_response = chat_completion_request(messages, functions)
    assistant_message = chat_response.json()["choices"][0]["message"]
    messages.append(assistant_message)
    if assistant_message.get("function_call"):
      results = execute_function_call(assistant_message)
      messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
      chat_response2 = chat_completion_request(messages)
      pretty_print_conversation(messages)
      messages.append(chat_response2.json()["choices"][0]["message"])
    pretty_print_conversation(messages)
    

In [12]:
queries = [
 "how many artists are there?",# y SELECT count(*) FROM artists
 "How many artists feature Santana?",
  "What  genre has the most tracks?",
  "What genre has the least tracks?",
  "What tracks are the most purchased",## ignores ties, uses duplicated
  "What artist has made the most albums",
  "List the 5 highest spending customers?",
  "Which customer has the most genres of music",
  "Which customer has the most eclectic taste in music",
]

In [13]:
do_conversation(conn, queries[7])

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Which customer has the most genres of music

assistant: {'name': 'ask_database', 'arguments': '{\n"query": "SELECT c.FirstName, c.LastName, COUNT(DISTINCT t.GenreId) as GenreCount FROM customers c JOIN invoices i ON c.CustomerId=i.CustomerId JOIN invoice_items ii ON i.InvoiceId=ii.InvoiceId JOIN tracks t ON ii.TrackId=t.TrackId GROUP BY c.CustomerId ORDER BY GenreCount DESC LIMIT 1"\n}'}

function (ask_database): [('Luis', 'Rojas', 12)]

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Which customer has the most genres of music

assistant: {'name': 'ask_database', 'arguments': '{\n"query": "SELECT c.FirstName, c.LastName, COUNT(DISTINCT t.GenreId) as GenreCount FROM customers c JOIN invoices i ON c.CustomerId=i.CustomerId JOIN invoice_items ii ON i.InvoiceId=ii.InvoiceId JOIN tracks t ON ii.TrackId=t.TrackId GROUP BY c.CustomerId ORDER BY

In [ ]:
do_conversation(conn, queries[1])

## Next Steps

See our other [notebook](How_to_call_functions_for_knowledge_retrieval.ipynb) that demonstrates how to use the Chat Completions API and functions for knowledge retrieval to interact conversationally with a knowledge base.